<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/day3_sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuración

Comienza instalando e importando el SDK de la API de Gemini en Python.


In [1]:
%pip install -q -U 'google-generativeai>=0.8.3'

In [2]:
import google.generativeai as genai

Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada como un secreto en Kaggle llamado GOOGLE_API_KEY.

Si aún no tienes una clave API, puedes obtener una en AI Studio. Consulta las instrucciones detalladas en la documentación.

Para que la clave esté disponible a través de los secretos de Kaggle, selecciona "Secrets" en el menú "Add-ons" y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.


In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


In [3]:
genai.configure(api_key='')

Crea una base de datos local

En este ejemplo básico, crearás una base de datos SQLite local y agregarás datos sintéticos para tener algo con lo que hacer consultas.

Carga la extensión de SQL en IPython para que puedas interactuar con la base de datos usando comandos mágicos (%).


In [4]:
%load_ext sql
%sql sqlite:///sample.db


Crea las tablas e inserta algunos datos sintéticos. Si lo deseas, puedes modificar la estructura y los datos.

In [5]:
%%sql
-- Crea la tabla 'productos'
CREATE TABLE IF NOT EXISTS productos (
  	producto_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	nombre_producto VARCHAR(255) NOT NULL,
  	precio DECIMAL(10, 2) NOT NULL
  );

-- Crea la tabla 'personal'
CREATE TABLE IF NOT EXISTS personal (
  	empleado_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	nombre VARCHAR(255) NOT NULL,
  	apellido VARCHAR(255) NOT NULL
  );

-- Crea la tabla 'ordenes'
CREATE TABLE IF NOT EXISTS ordenes (
  	orden_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	nombre_cliente VARCHAR(255) NOT NULL,
  	empleado_id INTEGER NOT NULL,
  	producto_id INTEGER NOT NULL,
  	FOREIGN KEY (empleado_id) REFERENCES personal (empleado_id),
  	FOREIGN KEY (producto_id) REFERENCES productos (producto_id)
  );

-- Inserta datos en la tabla 'productos'
INSERT INTO productos (nombre_producto, precio) VALUES
  	('Laptop', 799.99),
  	('Teclado', 129.99),
  	('Mouse', 29.99);

-- Inserta datos en la tabla 'personal'
INSERT INTO personal (nombre, apellido) VALUES
  	('Alicia', 'Pérez'),
  	('Roberto', 'Gómez'),
  	('Carlos', 'Ramírez');

-- Inserta datos en la tabla 'ordenes'
INSERT INTO ordenes (nombre_cliente, empleado_id, producto_id) VALUES
  	('David Lee', 1, 1),
  	('Emilia Chen', 2, 2),
  	('Francisco Brown', 1, 3);


 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

Define funciones para interactuar con la base de datos

Las llamadas a funciones en el SDK de Python de la API de Gemini pueden implementarse definiendo un esquema OpenAPI que se pasa al modelo. Alternativamente, puedes definir funciones en Python y permitir que el SDK inspeccione estas funciones para definir automáticamente el esquema.

Es importante que las funciones tengan anotaciones de tipo y docstrings precisas, ya que el modelo no puede inspeccionar el cuerpo de la función; la documentación actúa como la interfaz.

Proporcionaremos tres funcionalidades clave: listar tablas, describir una tabla y ejecutar una consulta. Esto permitirá al modelo (o a otro usuario) entender e interrogar la base de datos.

Primero, crea una conexión de base de datos que será utilizada por todas las funciones.


In [6]:
import sqlite3

archivo_bd = "sample.db"
conexion_bd = sqlite3.connect(archivo_bd)


La primera función listará todas las tablas disponibles en la base de datos. Defínela y pruébala para asegurarte de que funciona.

In [7]:
def listar_tablas() -> list[str]:
    """Obtén los nombres de todas las tablas en la base de datos."""
    print(' - Llamada a BD: listar_tablas')
    cursor = conexion_bd.cursor()
    # Obtén los nombres de las tablas.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = cursor.fetchall()
    return [t[0] for t in tablas]
listar_tablas()


 - Llamada a BD: listar_tablas


['productos', 'sqlite_sequence', 'personal', 'ordenes']

Una vez que se conocen las tablas disponibles, el siguiente paso es entender qué columnas están disponibles en una tabla dada. Define esta función y pruébala.

In [8]:
def describir_tabla(nombre_tabla: str) -> list[tuple[str, str]]:
    """Consulta el esquema de la tabla.
    Retorna:
      Lista de columnas, donde cada entrada es una tupla de (columna, tipo).
    """
    print(' - Llamada a BD: describir_tabla')
    cursor = conexion_bd.cursor()
    cursor.execute(f"PRAGMA table_info({nombre_tabla});")
    esquema = cursor.fetchall()
    # [índice de columna, nombre de columna, tipo de columna, ...]
    return [(col[1], col[2]) for col in esquema]
describir_tabla("productos")


 - Llamada a BD: describir_tabla


[('producto_id', 'INTEGER'),
 ('nombre_producto', 'VARCHAR(255)'),
 ('precio', 'DECIMAL(10, 2)')]

Ahora que el sistema sabe qué tablas y columnas están presentes, tiene suficiente información para generar y ejecutar una consulta SELECT. Define esta funcionalidad y pruébala.

In [9]:
def ejecutar_consulta(sql: str) -> list[list[str]]:
    """Ejecuta una instrucción SELECT, retornando los resultados."""
    print(' - Llamada a BD: ejecutar_consulta')
    cursor = conexion_bd.cursor()
    cursor.execute(sql)
    return cursor.fetchall()
ejecutar_consulta("SELECT * FROM productos")


 - Llamada a BD: ejecutar_consulta


[(1, 'Laptop', 799.99), (2, 'Teclado', 129.99), (3, 'Mouse', 29.99)]

Implementar llamadas a funciones

Ahora puedes combinar todo esto en una llamada a la API de Gemini.

La funcionalidad de llamadas a funciones funciona añadiendo mensajes específicos a una sesión de chat. Cuando se definen esquemas de funciones y están disponibles para el modelo, en lugar de devolver una respuesta en texto, el modelo puede devolver una llamada a función (`function_call`). Cuando esto sucede, el cliente debe responder con una respuesta de función (`function_response`), permitiendo que la conversación continúe normalmente.

El SDK también admite llamadas automáticas a funciones, donde las funciones definidas se invocan automáticamente. Esto es útil pero debe exponerse solo cuando sea seguro, como cuando las funciones no tienen efectos secundarios.


In [10]:
# Estas son las funciones definidas anteriormente.
herramientas_bd = [listar_tablas, describir_tabla, ejecutar_consulta]
instruccion = """Eres un chatbot útil que puede interactuar con una base de datos SQL para una tienda de computadoras.
Toma las preguntas de los usuarios y conviértelas en consultas SQL usando las herramientas disponibles.
Una vez que tengas la información que necesitas, responde la pregunta del usuario utilizando los datos devueltos.
Usa listar_tablas para ver las tablas disponibles, describir_tabla para entender el esquema y ejecutar_consulta para emitir consultas SELECT en SQL."""
modelo = genai.GenerativeModel(
    "models/gemini-1.5-flash-latest", tools=herramientas_bd, system_instruction=instruccion
)


Ahora puedes iniciar una conversación en el chat y hacer preguntas sobre el contenido de la base de datos.

In [16]:
from google.api_core import retry

In [17]:
# Define la política de reintento
retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

In [18]:
# Inicia el chat
chat = modelo.start_chat(enable_automatic_function_calling=True)

In [19]:
# Envía un mensaje al modelo
respuesta = chat.send_message("¿Cuál es el producto más barato?", request_options=retry_policy)
print(respuesta.text)

 - Llamada a BD: listar_tablas
 - Llamada a BD: describir_tabla
 - Llamada a BD: ejecutar_consulta
El producto más barato es el Mouse.



In [20]:
chat = modelo.start_chat(enable_automatic_function_calling=True)
respuesta = chat.send_message("¿Cuál es el producto más barato?", request_options={"retry": retry.Retry(predicate=retry.if_transient_error)})
print(respuesta.text)


 - Llamada a BD: listar_tablas
 - Llamada a BD: describir_tabla
 - Llamada a BD: ejecutar_consulta
El producto más barato es el Mouse.



Continúa la conversación haciendo una pregunta de seguimiento.

In [21]:
respuesta = chat.send_message("¿Y cuánto cuesta?", request_options={"retry": retry.Retry(predicate=retry.if_transient_error)})
print(respuesta.text)


 - Llamada a BD: ejecutar_consulta
El Mouse cuesta $29.99.



Explora la sesión de chat y haz tus propias preguntas.

In [22]:
modelo_pro = genai.GenerativeModel(
    "models/gemini-1.5-pro-latest", tools=herramientas_bd, system_instruction=instruccion
)
chat_pro = modelo_pro.start_chat(enable_automatic_function_calling=True)
respuesta = chat_pro.send_message('¿Qué vendedor vendió el producto más barato?', request_options={"retry": retry.Retry(predicate=retry.if_transient_error)})
print(respuesta.text)


 - Llamada a BD: listar_tablas
 - Llamada a BD: describir_tabla
 - Llamada a BD: describir_tabla
 - Llamada a BD: describir_tabla
 - Llamada a BD: ejecutar_consulta


Alicia Pérez vendió el producto más barato.


Inspecciona la conversación

Puedes inspeccionar las llamadas que realiza el modelo y lo que el cliente devuelve en respuesta usando `chat.history`.


In [23]:
import textwrap

In [24]:
def imprimir_historial_chat(chat):
    """Imprime cada turno en el historial del chat, incluyendo llamadas a funciones y respuestas."""
    for evento in chat.history:
        print(f"{evento.role.capitalize()}:")
        for parte in evento.parts:
            if txt := parte.text:
                print(f'  "{txt}"')
            elif fn := parte.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Llamada a función: {fn.name}({args})")
            elif resp := parte.function_response:
                print("  Respuesta de función:")
                print(textwrap.indent(str(resp), "    "))
        print()
imprimir_historial_chat(chat)


User:
  "¿Cuál es el producto más barato?"

Model:
  Llamada a función: listar_tablas()

User:
  Respuesta de función:
    name: "listar_tablas"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              string_value: "productos"
            }
            values {
              string_value: "sqlite_sequence"
            }
            values {
              string_value: "personal"
            }
            values {
              string_value: "ordenes"
            }
          }
        }
      }
    }


Model:
  Llamada a función: describir_tabla(nombre_tabla=productos)

User:
  Respuesta de función:
    name: "describir_tabla"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              list_value {
                values {
                  string_value: "producto_id"
                }
                values {
                  string_value: "INTEGER"
 

Lecturas adicionales

Para aprender más sobre lo que puede hacer la API de Gemini con llamadas a funciones, consulta el libro de recetas sobre llamadas a funciones (Function calling
